Content:
* [Introduction](#1):
* [Import Data Set](#2):
* [Preparing Images](#3):
* [Normalize the Data](#4):
* [Label Encoding](#5):
* [Implementing with Keras](#6):
* [Set the Optimizer and Annealer](#7):
* [Compile Model](#8):
* [Data Augmentation](#9):
* [Epochs and Batch Size](#10):
* [Fit the Model](#11):
* [Evaluate the model](#12):
* [Predict Test Data](#13):

<a id="1"></a> <br>
# INTRODUCTION

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


import os
print(os.listdir("../input"))

import warnings
warnings.filterwarnings('ignore')


<a id="2"></a> <br>
## Import Dataset

In [ ]:
train = pd.read_csv("../input/train.csv")

In [ ]:
train.info()

In [ ]:
train.shape

In [ ]:
train.Id.describe()

In [ ]:
train.head(10)

In [ ]:
train.tail(10)

In [ ]:
y_train = train["Id"]
X_train = train.drop(labels = ["Id"], axis = 1)
y_train.head()

In [ ]:
train.isnull().sum().sum()

<a id="3"></a> <br>
## Preparing Images

In [ ]:
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

def prepareImages(train, shape, path):
    
    x_train = np.zeros((shape, 100, 100, 3))
    count = 0
    
    for fig in train['Image']:
        
       
        img = image.load_img("../input/"+path+"/"+fig, target_size=(100, 100, 3))
        x = image.img_to_array(img)
        x = preprocess_input(x)

        x_train[count] = x
        if (count%500 == 0):
            print("Processing image: ", count+1, ", ", fig)
        count += 1
    
    return x_train

In [ ]:
x_train = prepareImages(train, train.shape[0], "train")

<a id="4"></a> <br>
### Normalize the Data

In [ ]:
x_train = x_train / 255.0
print("x_train shape: ",x_train.shape)

In [ ]:
plt.imshow(x_train[0][:,:,0], cmap="gray")
plt.title(plt.title(train.iloc[0,0]))
plt.axis("off")
plt.show()

In [ ]:
plt.imshow(x_train[25360][:,:,0], cmap="gray")
plt.title(plt.title(train.iloc[25360,0]))
plt.axis("off")
plt.show()

In [ ]:
plt.imshow(x_train[55][:,:,0], cmap="gray")
plt.title(plt.title(train.iloc[55,0]))
plt.axis("off")
plt.show()

<a id="5"></a> <br>
## Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [ ]:
y_train = label_encoder.fit_transform(y_train)

In [ ]:
y_train[0:10]  

In [ ]:
y_train.shape

In [ ]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes = 5005)

In [ ]:
print(y_train.shape)
y_train 

<a id="6"></a> <br>
## Implementing with Keras

In [ ]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential # to create a cnn model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (5,5), padding = 'Same', activation = 'relu', input_shape = (100,100,3)))
model.add(Conv2D(filters = 16, kernel_size = (5,5), padding = 'Same', activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(Conv2D(filters = 32, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1], activation = "softmax"))

In [ ]:
model.summary()

<a id="7"></a> <br>
### Set the Optimizer and Annealer

In [ ]:
optimizer = Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999)

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

<a id="8"></a> <br>
### Compile Model

In [ ]:
model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])

<a id="10"></a> <br>
### Epochs and Batch Size

In [ ]:
epochs = 100
batch_size = 1000

<a id="11"></a>
### Fit the Model

In [ ]:
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[learning_rate_reduction])

<a id="12"></a> <br>
### Evaluate the model

In [ ]:
plt.plot(history.history['loss'], color='r', label="Train Loss")
plt.title("Train Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['acc'], color='g', label="Train Accuracy")
plt.title("Train Accuracy")
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
print('Train accuracy of the model: ',history.history['acc'][-1])

In [ ]:
print('Train loss of the model: ',history.history['loss'][-1])

<a id="13"></a> <br>
## Predict Test Data

In [ ]:
test = os.listdir("../input/test/")
print(len(test))

In [ ]:
col = ['Image']
test_data = pd.DataFrame(test, columns=col)
test_data['Id'] = ''

In [ ]:
x_test = prepareImages(test_data, test_data.shape[0], "test")
x_test /= 255

In [ ]:
predictions = model.predict(np.array(x_test), verbose=1)

In [ ]:
for i, pred in enumerate(predictions):
    test_data.loc[i, 'Id'] = ' '.join(label_encoder.inverse_transform(pred.argsort()[-5:][::-1]))

In [ ]:
test_data.head(10)
test_data.to_csv('submission_3.csv', index=False)